In [1]:
def collatz(n):
    if n % 2 == 0:
        return n/2
    else:
        return (3*n+1)

In [2]:
def collatz_steps(n, record_steps=False):
    steps = 0
    step_values = []
    while True:
        n = collatz(n)
        steps += 1
        if record_steps: # stupidly inefficient but it doesn't matter
            step_values.append(n)
        if n == 1:
            break
    if record_steps:
        return steps, step_values
    return steps

Data collected from: http://www.ericr.nl/wondrous/delrecs.html

In [11]:
import pandas as pd
from numpy import longdouble

delay_records = pd.read_csv('collatz delay records.csv')
delay_records.N = delay_records.N.str.replace(',', '').astype(longdouble)

In [12]:
delay_records['my_delay_calc'] = delay_records.N.apply(collatz_steps)
delay_records['difference'] = delay_records['Delay'] - delay_records['my_delay_calc']

In [41]:
delay_records.to_csv('collatz delay records with my calc.csv', index=False)

Lowest N with a delay value mismatch

In [5]:
lowest_N = delay_records[delay_records.difference != 0].N.sort_values().iloc[0]
print(lowest_N)

2081751768559.0


In [6]:
steps, step_values = collatz_steps(lowest_N, record_steps=True)
print(steps)

385


Obtained by inputing lowest_N at this page: http://www.ericr.nl/wondrous/showsteps.html

In [35]:
from numpy import longdouble, longlong, uint64

steps_comparison = pd.read_csv('collatz steps for 2081751768559.csv')#, converters={'Number': uint64})
# steps_comparison.Number = steps_comparison.Number.astype(longdouble)

File is not being read properly. This should end with '358' instead of '350'.

In [36]:
steps_comparison.Number.shift(-1).loc[65]

'1,039,526,319,483,350'

In [23]:
from numpy import iinfo, finfo, uint64

iinfo(uint64).max#, iinfo(longlong).max

18446744073709551615

In [8]:
step_value_differences = steps_comparison.Number.shift(-1) - pd.Series(step_values)
step_value_differences[step_value_differences != 0]

65     -8.0
67     -8.0
69     -8.0
70     -9.0
71     -8.0
       ... 
1433    NaN
1434    NaN
1435    NaN
1436    NaN
1437    NaN
Length: 1371, dtype: float64

So difference starts at step 65

In [10]:
step_values[65], steps_comparison.Number.shift(-1).loc[65]

(1039526319483358.0, 1039526319483350.0)

In [84]:
pd.DataFrame(step_values).merge(steps_comparison[['Number']].shift(-1), how='right', right_index=True, left_index=True).\
rename(columns={0: 'Calculated', 'Number': 'Downloaded'}).to_csv('calc vs downloaded collatz steps for 2081751768559.csv')